⚠️ If the generated code snippets do not work, please open an issue https://issues.hubocean.earth/form/f57a63fa-20d2-4268-af79-41eca8f374cd 🙏

In [ ]:
!pip install -U odp-sdk --quiet
!pip install -U pydeck --quiet && pip list | grep pydeck
!pip install -U geopandas --quiet && pip list | grep geopandas

In [ ]:
# Standard library imports
import json
import pandas as pd
import geojson

import shapely
from shapely import wkt
from shapely.geometry import box, MultiPolygon, Polygon
import geopandas as gpd

from odp.client import OdpClient  # The SDK

In [ ]:
client = OdpClient()

In [ ]:
## Request the dataset from the catalog using the UUID:
proSeas_dataset = client.catalog.get(("3e32fd06-4eb7-4da2-9acb-dd0ecb58aa88"))
proSeas_dataset.metadata.display_name

In [ ]:
proSeas_data = client.table_v2(proSeas_dataset)

## Create a geographic query and search for specific types of MPAs

In [ ]:
## Bounding box for Southern Norway
## Enter min and max latitude and longitude values to create a bounding box polygon below. Or use the structure below to add any Well-Known-Text or GeoJSON defined polygon.
lat_min = 56
lat_max = 62
lon_min = 8
lon_max = 12
query_geometry = box(lon_min, lat_min, lon_max, lat_max).wkt
query_geometry

### Fetch data from ODP 

In [ ]:
# Find Natura2000 sites within the search area
df_geo = pd.concat(proSeas_data.select(f"geometry within '{query_geometry}' AND boundary_source == 'Natura2000'").dataframes(), ignore_index=True)
df_geo.head()

## Export the data in your prefered file format

### Pandas

In [ ]:
# Uncomment the line below to select the output format

# Export to CSV
df_geo.to_csv('df_country.csv', index=False)

# Export to JSON
# df_country.to_json('df_country.json', orient='records', lines=True)

# Export to Parquet
# df_country.to_parquet('df_country.parquet', index=False)

### GeoPandas

In [ ]:
# Function to convert WKT string to 2D Shapely geometry
def convert_wkt_to_2d(geometry_wkt):
    if not geometry_wkt:  # Check if it's None or empty
        return None  

    try:
        geom = wkt.loads(geometry_wkt)  # Convert text to Shapely geometry
        if geom and hasattr(geom, "has_z") and geom.has_z:  # Ensure geom exists and has Z
            # Remove Z by keeping only X, Y coordinates
            return MultiPolygon([
                Polygon([(x, y) for x, y, *_ in polygon.exterior.coords])
                for polygon in geom.geoms
            ]) if isinstance(geom, MultiPolygon) else Polygon([(x, y) for x, y, *_ in geom.exterior.coords])
        
        return geom  # Already 2D
    except Exception as e:
        print(f"Error converting geometry: {e}")
        return None  # Return None instead of breaking

In [ ]:
# Convert DataFrame to GeoDataFrame
df_geo['geometry'] = df_geo['geometry'].apply(convert_wkt_to_2d)
gdf = gpd.GeoDataFrame(df_geo, geometry='geometry', crs="EPSG:4326")

In [ ]:
gdf.to_file("df_geo.geojson", driver="GeoJSON")
print("GeoJSON saved successfully!")